Como primer paso se configura una base de datos Postgres en Docker con el siguiente DDL empleando Dbeaver:

-- Tabla: Usuarios
CREATE TABLE Usuarios (
    UsuarioID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Apellido VARCHAR(100) NOT NULL,
    DNI VARCHAR(20) UNIQUE NOT NULL,
    Email VARCHAR(255) UNIQUE NOT NULL,
    Contraseña VARCHAR(255) NOT NULL,
    FechaRegistro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Categorías
CREATE TABLE Categorias (
    CategoriaID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Descripcion VARCHAR(255)
);

-- Tabla: Productos
CREATE TABLE Productos (
    ProductoID SERIAL PRIMARY KEY,
    Nombre VARCHAR(255) NOT NULL,
    Descripcion TEXT,
    Precio DECIMAL(10,2) NOT NULL,
    Stock INT NOT NULL,
    CategoriaID INT REFERENCES Categorias(CategoriaID)
);

-- Tabla: Órdenes
CREATE TABLE Ordenes (
    OrdenID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    FechaOrden TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    Total DECIMAL(10,2) NOT NULL,
    Estado VARCHAR(50) DEFAULT 'Pendiente'
);

-- Tabla: Detalle de Órdenes
CREATE TABLE DetalleOrdenes (
    DetalleID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Cantidad INT NOT NULL,
    PrecioUnitario DECIMAL(10,2) NOT NULL
);

-- Tabla: Direcciones de Envío
CREATE TABLE DireccionesEnvio (
    DireccionID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    Calle VARCHAR(255) NOT NULL,
    Ciudad VARCHAR(100) NOT NULL,
    Departamento VARCHAR(100),
    Provincia VARCHAR(100),
    Distrito VARCHAR(100),
    Estado VARCHAR(100),
    CodigoPostal VARCHAR(20),
    Pais VARCHAR(100) NOT NULL
);

-- Tabla: Carrito de Compras
CREATE TABLE Carrito (
    CarritoID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Cantidad INT NOT NULL,
    FechaAgregado TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Métodos de Pago
CREATE TABLE MetodosPago (
    MetodoPagoID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100) NOT NULL,
    Descripcion VARCHAR(255)
);

-- Tabla: Ordenes Métodos de Pago
CREATE TABLE OrdenesMetodosPago (
    OrdenMetodoID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    MetodoPagoID INT REFERENCES MetodosPago(MetodoPagoID),
    MontoPagado DECIMAL(10,2) NOT NULL
);

-- Tabla: Reseñas de Productos
CREATE TABLE ReseñasProductos (
    ReseñaID SERIAL PRIMARY KEY,
    UsuarioID INT REFERENCES Usuarios(UsuarioID),
    ProductoID INT REFERENCES Productos(ProductoID),
    Calificacion INT CHECK (Calificacion >= 1 AND Calificacion <= 5),
    Comentario TEXT,
    Fecha TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Tabla: Historial de Pagos
CREATE TABLE HistorialPagos (
    PagoID SERIAL PRIMARY KEY,
    OrdenID INT REFERENCES Ordenes(OrdenID),
    MetodoPagoID INT REFERENCES MetodosPago(MetodoPagoID),
    Monto DECIMAL(10,2) NOT NULL,
    FechaPago TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    EstadoPago VARCHAR(50) DEFAULT 'Procesando'
);

Carga de CSV: a partir de esta seccion s ecargan los csv

Instalar dependencias en el entorno

In [3]:
%pip install pandas sqlalchemy psycopg2-binary python-dotenv

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 2.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 2.0 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.6 MB 1.7 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.6 MB 1.8 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.6 MB 1.8 MB/s eta 0:00:01
   -------------------------------- ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Importar librerías

In [10]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, DECIMAL, TIMESTAMP, ForeignKey
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
import os

Conexion a la base de datos

In [11]:
load_dotenv()  # Carga las credenciales desde el archivo .env (seguridad)

# Variables para conectar con PostgreSQL
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = "localhost"
port = "5432"
database = "EcommerceDB"

# Crea el motor SQLAlchemy que sirve de puente entre Python y la BD
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

# Base es la clase madre para definir modelos ORM que se conectan con tablas
Base = declarative_base()

# Crea una fábrica de sesiones que permite interactuar con la BD (insertar, consultar, etc.)
Session = sessionmaker(bind=engine)
session = Session()


C:\Users\valentin.e.bussolini\AppData\Local\Temp\ipykernel_34248\4226455634.py:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Definir el modelo para cada una de las tablas

In [ ]:
# 🧍‍♂️ TABLA USUARIOS
class Usuario(Base):
    __tablename__ = 'usuarios'

    usuarioid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    apellido = Column(String(100), nullable=False)
    dni = Column(String(20), unique=True, nullable=False)
    email = Column(String(255), unique=True, nullable=False)
    contraseña = Column(String(255), nullable=False)
    fecharegistro = Column(TIMESTAMP)

# 🗂️ TABLA CATEGORÍAS
class Categoria(Base):
    __tablename__ = 'categorias'

    categoriaid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

# 📦 TABLA PRODUCTOS
class Producto(Base):
    __tablename__ = 'productos'

    productoid = Column(Integer, primary_key=True)
    nombre = Column(String(255), nullable=False)
    descripcion = Column(Text)
    precio = Column(DECIMAL(10,2), nullable=False)
    stock = Column(Integer, nullable=False)
    categoriaid = Column(Integer, ForeignKey('categorias.categoriaid'))

# 🧾 TABLA ÓRDENES
class Orden(Base):
    __tablename__ = 'ordenes'

    ordenid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    fechaorden = Column(TIMESTAMP)
    total = Column(DECIMAL(10,2), nullable=False)
    estado = Column(String(50))

# 📄 TABLA DETALLE DE ÓRDENES
class DetalleOrden(Base):
    __tablename__ = 'detalleordenes'

    detalleid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    cantidad = Column(Integer, nullable=False)
    preciounitario = Column(DECIMAL(10,2), nullable=False)

# 📍 TABLA DIRECCIONES DE ENVÍO
class DireccionEnvio(Base):
    __tablename__ = 'direccionesenvio'

    direccionid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    calle = Column(String(255), nullable=False)
    ciudad = Column(String(100), nullable=False)
    departamento = Column(String(100))
    provincia = Column(String(100))
    distrito = Column(String(100))
    estado = Column(String(100))
    codigopostal = Column(String(20))
    pais = Column(String(100), nullable=False)

# 🛒 TABLA CARRITO DE COMPRAS
class Carrito(Base):
    __tablename__ = 'carrito'

    carritoid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    cantidad = Column(Integer, nullable=False)
    fechaagregado = Column(TIMESTAMP)

# 💳 TABLA MÉTODOS DE PAGO
class MetodoPago(Base):
    __tablename__ = 'metodospago'

    metodopagoid = Column(Integer, primary_key=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

# 🧾 TABLA ÓRDENES-MÉTODOS DE PAGO
class OrdenMetodoPago(Base):
    __tablename__ = 'ordenesmetodospago'

    ordenmetodoid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'))
    montopagado = Column(DECIMAL(10,2), nullable=False)

# 🌟 TABLA RESEÑAS DE PRODUCTOS
class ReseñaProducto(Base):
    __tablename__ = 'reseñasproductos'

    reseñaid = Column(Integer, primary_key=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'))
    productoid = Column(Integer, ForeignKey('productos.productoid'))
    calificacion = Column(Integer)
    comentario = Column(Text)
    fecha = Column(TIMESTAMP)

# 🧾 TABLA HISTORIAL DE PAGOS
class HistorialPago(Base):
    __tablename__ = 'historialpagos'

    pagoid = Column(Integer, primary_key=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'))
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'))
    monto = Column(DECIMAL(10,2), nullable=False)
    fechapago = Column(TIMESTAMP)
    estadopago = Column(String(50))

insertar CSV en las tablas correspondientes

In [20]:
# Diccionario de mapeo entre archivos CSV y nombres de tabla en la BD (filename.csv, tablenameinDB)
csv_table_map = {
    "2.Usuarios.csv": "usuarios",
    "3.Categorias.csv": "categorias",
    "4.Productos.csv": "productos",
    "5.ordenes.csv": "ordenes",
    "6.detalle_ordenes.csv": "detalleordenes",
    "7.direcciones_envio.csv": "direccionesenvio",
    "8.carrito.csv": "carrito",
    "9.metodos_pago.csv": "metodospago",
    "10.ordenes_metodospago.csv": "ordenesmetodospago",
    "11.resenas_productos.csv": "reseñasproductos",
    "12.historial_pagos.csv": "historialpagos"
}

# Cargar cada archivo y escribir en su tabla
for csv_file, table_name in csv_table_map.items():
    file_path = os.path.join("..", "data", csv_file)
    try:
        df = pd.read_csv(file_path, encoding="utf-8-sig")  # se emplea utf-8-sig porque maneja la ñ y tildes correctamente

        # Validación: caso archivo vacío
        if df.empty:
            print(f"⚠️ '{csv_file}' está vacío. No se cargó ningún registro.")
            continue  # Salta al siguiente archivo

        # Normalizar encabezados (lowercase, sin espacios)
        df.columns = df.columns.str.lower().str.strip()

        print(f"▶️ Cargando '{csv_file}' en tabla '{table_name}'...")
        df.to_sql(table_name, engine, if_exists="append", index=False)
        print(f"✅ Tabla '{table_name}' cargada con {len(df)} registros.")
    except Exception as e:
        print(f"⚠️ Error al cargar '{csv_file}': {e}")


▶️ Cargando '2.Usuarios.csv' en tabla 'usuarios'...
✅ Tabla 'usuarios' cargada con 1000 registros.
▶️ Cargando '3.Categorias.csv' en tabla 'categorias'...
✅ Tabla 'categorias' cargada con 12 registros.
▶️ Cargando '4.Productos.csv' en tabla 'productos'...
✅ Tabla 'productos' cargada con 36 registros.
▶️ Cargando '5.ordenes.csv' en tabla 'ordenes'...
✅ Tabla 'ordenes' cargada con 10000 registros.
▶️ Cargando '6.detalle_ordenes.csv' en tabla 'detalleordenes'...
✅ Tabla 'detalleordenes' cargada con 10000 registros.
▶️ Cargando '7.direcciones_envio.csv' en tabla 'direccionesenvio'...
✅ Tabla 'direccionesenvio' cargada con 1000 registros.
▶️ Cargando '8.carrito.csv' en tabla 'carrito'...
✅ Tabla 'carrito' cargada con 5000 registros.
⚠️ '9.metodos_pago.csv' está vacío. No se cargó ningún registro.
▶️ Cargando '10.ordenes_metodospago.csv' en tabla 'ordenesmetodospago'...
✅ Tabla 'ordenesmetodospago' cargada con 10000 registros.
▶️ Cargando '11.resenas_productos.csv' en tabla 'reseñasproductos

OUTOUT DEL BLOQUE DE ARRIBA:

▶️ Cargando '2.Usuarios.csv' en tabla 'usuarios'...
✅ Tabla 'usuarios' cargada con 1000 registros.
▶️ Cargando '3.Categorias.csv' en tabla 'categorias'...
✅ Tabla 'categorias' cargada con 12 registros.
▶️ Cargando '4.Productos.csv' en tabla 'productos'...
✅ Tabla 'productos' cargada con 36 registros.
▶️ Cargando '5.ordenes.csv' en tabla 'ordenes'...
✅ Tabla 'ordenes' cargada con 10000 registros.
▶️ Cargando '6.detalle_ordenes.csv' en tabla 'detalleordenes'...
✅ Tabla 'detalleordenes' cargada con 10000 registros.
▶️ Cargando '7.direcciones_envio.csv' en tabla 'direccionesenvio'...
✅ Tabla 'direccionesenvio' cargada con 1000 registros.
▶️ Cargando '8.carrito.csv' en tabla 'carrito'...
✅ Tabla 'carrito' cargada con 5000 registros.
⚠️ '9.metodos_pago.csv' está vacío. No se cargó ningún registro.
▶️ Cargando '10.ordenes_metodospago.csv' en tabla 'ordenesmetodospago'...
✅ Tabla 'ordenesmetodospago' cargada con 10000 registros.
▶️ Cargando '11.resenas_productos.csv' en tabla 'reseñasproductos'...
✅ Tabla 'reseñasproductos' cargada con 7172 registros.
▶️ Cargando '12.historial_pagos.csv' en tabla 'historialpagos'...
✅ Tabla 'historialpagos' cargada con 10000 registros.

Análisis de los datos -- NULOS

In [22]:
#Funcion para identificar nulls, soft nulls y porcentajes

weird_nan_representations = ["NA", "N/A", "null", "NULL", "nan", "NaN", "", " ", "?"]

def analizar_nulos(tabla):
    print(f"🔎 Análisis de nulos para: {tabla}")
    df = pd.read_sql(f"SELECT * FROM {tabla}", engine)

    total_registros = len(df)

    # Nulos estándar
    nulos_abs = df.isnull().sum()
    nulos_pct = round(df.isnull().mean() * 100, 2)

    # Weird NaNs
    weird_counts = {}
    for col in df.columns:
        if df[col].dtype == "object":
            col_lower = df[col].astype(str).str.strip().str.lower()
            weird_counts[col] = col_lower.isin([w.lower() for w in weird_nan_representations]).sum()
        else:
            weird_counts[col] = 0

    weird_pct = {col: round(weird_counts[col] / total_registros * 100, 2) for col in df.columns}

    # Totales combinados
    total_nulos = nulos_abs + pd.Series(weird_counts)
    total_pct = round(total_nulos / total_registros * 100, 2)

    resultado = pd.DataFrame({
        "Nulos estándar": nulos_abs,
        "Nulos estándar (%)": nulos_pct,
        "Nulos no reconocidos": pd.Series(weird_counts),
        "Nulos no reconocidos (%)": pd.Series(weird_pct),
        "Total nulos": total_nulos,
        "Porcentaje total (%)": total_pct
    })

    print(resultado.sort_values(by="Total nulos", ascending=False))
    print("-" * 50)

In [27]:
for table_name in csv_table_map.values():
    try:
        analizar_nulos(table_name)
    except Exception as e:
        print(f"⚠️ La tabla '{table_name}' no pudo ser analizada: {e}")
        print("-" * 50)

🔎 Análisis de nulos para: usuarios
            Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
nombre                   0                 0.0                     0   
apellido                 0                 0.0                     0   
dni                      0                 0.0                     0   
email                    0                 0.0                     0   
contraseña               0                 0.0                     0   

            Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
nombre                           0.0            0                   0.0  
apellido                         0.0            0                   0.0  
dni                              0.0            0                   0.0  
email                            0.0            0                   0.0  
contraseña                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: categorias


OUTPUT DEL BLOQUE DE ARRIBA:

🔎 Análisis de nulos para: usuarios
            Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
nombre                   0                 0.0                     0   
apellido                 0                 0.0                     0   
dni                      0                 0.0                     0   
email                    0                 0.0                     0   
contraseña               0                 0.0                     0   

            Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
nombre                           0.0            0                   0.0  
apellido                         0.0            0                   0.0  
dni                              0.0            0                   0.0  
email                            0.0            0                   0.0  
contraseña                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: categorias
             Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
nombre                    0                 0.0                     0   
descripcion               0                 0.0                     0   

             Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
nombre                            0.0            0                   0.0  
descripcion                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: productos
             Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
nombre                    0                 0.0                     0   
descripcion               0                 0.0                     0   
precio                    0                 0.0                     0   
stock                     0                 0.0                     0   
categoriaid               0                 0.0                     0   

             Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
nombre                            0.0            0                   0.0  
descripcion                       0.0            0                   0.0  
precio                            0.0            0                   0.0  
stock                             0.0            0                   0.0  
categoriaid                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: ordenes
            Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
usuarioid                0                 0.0                     0   
fechaorden               0                 0.0                     0   
total                    0                 0.0                     0   
estado                   0                 0.0                     0   

            Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
usuarioid                        0.0            0                   0.0  
fechaorden                       0.0            0                   0.0  
total                            0.0            0                   0.0  
estado                           0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: detalleordenes
                Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
ordenid                      0                 0.0                     0   
productoid                   0                 0.0                     0   
cantidad                     0                 0.0                     0   
preciounitario               0                 0.0                     0   

                Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
ordenid                              0.0            0                   0.0  
productoid                           0.0            0                   0.0  
cantidad                             0.0            0                   0.0  
preciounitario                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: direccionesenvio
              Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
usuarioid                  0                 0.0                     0   
calle                      0                 0.0                     0   
ciudad                     0                 0.0                     0   
departamento               0                 0.0                     0   
provincia                  0                 0.0                     0   
distrito                   0                 0.0                     0   
estado                     0                 0.0                     0   
codigopostal               0                 0.0                     0   
pais                       0                 0.0                     0   

              Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
usuarioid                          0.0            0                   0.0  
calle                              0.0            0                   0.0  
ciudad                             0.0            0                   0.0  
departamento                       0.0            0                   0.0  
provincia                          0.0            0                   0.0  
distrito                           0.0            0                   0.0  
estado                             0.0            0                   0.0  
codigopostal                       0.0            0                   0.0  
pais                               0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: carrito
               Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
usuarioid                   0                 0.0                     0   
productoid                  0                 0.0                     0   
cantidad                    0                 0.0                     0   
fechaagregado               0                 0.0                     0   

               Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
usuarioid                           0.0            0                   0.0  
productoid                          0.0            0                   0.0  
cantidad                            0.0            0                   0.0  
fechaagregado                       0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: metodospago
⚠️ La tabla 'metodospago' no pudo ser analizada: (psycopg2.errors.UndefinedTable) relation "metodospago" does not exist
LINE 1: SELECT * FROM metodospago
                      ^

[SQL: SELECT * FROM metodospago]
(Background on this error at: https://sqlalche.me/e/20/f405)
--------------------------------------------------
🔎 Análisis de nulos para: ordenesmetodospago
              Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
ordenid                    0                 0.0                     0   
metodopagoid               0                 0.0                     0   
montopagado                0                 0.0                     0   

              Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
ordenid                            0.0            0                   0.0  
metodopagoid                       0.0            0                   0.0  
montopagado                        0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: reseñasproductos
              Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
usuarioid                  0                 0.0                     0   
productoid                 0                 0.0                     0   
calificacion               0                 0.0                     0   
comentario                 0                 0.0                     0   
fecha                      0                 0.0                     0   

              Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
usuarioid                          0.0            0                   0.0  
productoid                         0.0            0                   0.0  
calificacion                       0.0            0                   0.0  
comentario                         0.0            0                   0.0  
fecha                              0.0            0                   0.0  
--------------------------------------------------
🔎 Análisis de nulos para: historialpagos
              Nulos estándar  Nulos estándar (%)  Nulos no reconocidos  \
ordenid                    0                 0.0                     0   
metodopagoid               0                 0.0                     0   
monto                      0                 0.0                     0   
fechapago                  0                 0.0                     0   
estadopago                 0                 0.0                     0   

              Nulos no reconocidos (%)  Total nulos  Porcentaje total (%)  
ordenid                            0.0            0                   0.0  
metodopagoid                       0.0            0                   0.0  
monto                              0.0            0                   0.0  
fechapago                          0.0            0                   0.0  
estadopago                         0.0            0                   0.0  
--------------------------------------------------